# Vanna.AI Test

https://vanna.ai/docs/snowflake-openai-vanna-vannadb.html

In [1]:
import os
import vanna
from vanna.remote import VannaDefault

In [2]:
SNOW_ACCOUNT = os.environ.get("SNOW_ACCOUNT")
SNOW_USER = os.environ.get("SNOW_USER")
SNOW_PASS = os.environ.get("SNOW_PASS")
SNOW_WAREHOUSE = "DEADPOOL"
SNOW_DATABASE = "DEADPOOL"

api_key = vanna.get_api_key("VANNA_API_KEY")

In [3]:
vanna_model_name = "deadpool-test"
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [4]:
snow_account = os.environ.get("VANNA_API_KEY")

vn.connect_to_snowflake(
    account=SNOW_ACCOUNT,
    username=SNOW_USER,
    password=SNOW_PASS,
    database=SNOW_DATABASE,
    role="ENGINEER",
)

## Only Run This Next Section if There are Changes

In [5]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE (TABLE_NAME = 'PLAYERS' OR TABLE_NAME = 'PICKS' OR TABLE_NAME = 'SCORES') AND TABLE_SCHEMA = 'PROD'")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

In [6]:
plan

Train on Information Schema: DEADPOOL.PROD PLAYERS
Train on Information Schema: DEADPOOL.PROD SCORES
Train on Information Schema: DEADPOOL.PROD PICKS

In [9]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
	CREATE OR REPLACE TABLE DEADPOOL.PROD.PICKS (
		NAME VARCHAR(256),
		BIRTH_DATE DATE,
		DEATH_DATE DATE,
		PICKED_BY VARCHAR(256),
		WIKI_PAGE VARCHAR(256),
		YEAR NUMBER(38,0),
		TIMESTAMP TIMESTAMP_NTZ(9),
		WIKI_ID VARCHAR(256),
		AGE NUMBER(10,1),
		FOREIGN KEY (PICKED_BY) REFERENCES DEADPOOL.PROD.PLAYERS(ID)
);
""")

vn.train(ddl="""
	CREATE OR REPLACE TABLE DEADPOOL.PROD.PLAYERS (
		FIRST_NAME VARCHAR(256),
		LAST_NAME VARCHAR(256),
		EMAIL VARCHAR(256),
		OPT_IN BOOLEAN,
		SMS VARCHAR(256),
		ID VARCHAR(36),
		PASSWORD VARCHAR(256),
		YEAR_ONE NUMBER(10,1),
		YEAR_TWO NUMBER(10,1),
		PRIMARY KEY (ID),
);
""")

vn.train(ddl="""
	CREATE OR REPLACE TRANSIENT TABLE DEADPOOL.PROD.SCORES (
		PLAYER_ID VARCHAR(36),
		TOTAL NUMBER(38,0),
		BASE_SCORE NUMBER(24,1),
		FIRST_BLOOD NUMBER(38,0),
		LAST_BLOOD NUMBER(38,0),
		Q_ONE NUMBER(38,0),
		Q_TWO NUMBER(38,0),
		Q_THREE NUMBER(38,0),
		Q_FOUR NUMBER(38,0),
  		foreign key (PICKED_BY) references DEADPOOL.PROD.PLAYERS(ID)
);
);
""")

# Add documentation about your business terminology or definitions.
vn.train(documentation="You are the Arbiter.  You are the judge of the game DEADPOOL. You have access to a data frame with a column called NAME, which represents celebrity picks for this year, 2024.  The PLAYERS column contains the game's participants.")  

vn.train(documentation="When asked about the score or points in the game, you should join the table SCORES on SCORES.ID = PLAYERS.ID. This will allow you certain different values, but the TOTAL column is the value you should retreive.")  

vn.train(documentation="When you deal with these dataframes, please join them together by the ID field in PLAYERS and the PICKED_BY field in PICKS.")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="""
	SELECT CONCAT(FIRST_NAME || ' ' || LAST_NAME) AS PLAYER_NAME, COUNT(NAME) AS TOTAL_PICKS 
	FROM PICKS
	LEFT JOIN PLAYERS
	ON PICKS.PICKED_BY = PLAYERS.ID
	WHERE YEAR = 2024 
	GROUP BY 1
""")

vn.train(sql="""
	SELECT CONCAT(FIRST_NAME, ' ', LAST_NAME) AS PLAYER_NAME, TOTAL
	FROM DEADPOOL.PROD.PLAYERS
	JOIN DEADPOOL.PROD.SCORES
	ON PLAYERS.ID = SCORES.ID
	ORDER BY TOTAL DESC;         
""")

Adding ddl: 
	CREATE OR REPLACE TABLE DEADPOOL.PROD.PICKS (
		NAME VARCHAR(256),
		BIRTH_DATE DATE,
		DEATH_DATE DATE,
		PICKED_BY VARCHAR(256),
		WIKI_PAGE VARCHAR(256),
		YEAR NUMBER(38,0),
		TIMESTAMP TIMESTAMP_NTZ(9),
		WIKI_ID VARCHAR(256),
		AGE NUMBER(10,1),
		FOREIGN KEY (PICKED_BY) REFERENCES DEADPOOL.PROD.PLAYERS(ID)
);

Adding ddl: 
	CREATE OR REPLACE TABLE DEADPOOL.PROD.PLAYERS (
		FIRST_NAME VARCHAR(256),
		LAST_NAME VARCHAR(256),
		EMAIL VARCHAR(256),
		OPT_IN BOOLEAN,
		SMS VARCHAR(256),
		ID VARCHAR(36),
		PASSWORD VARCHAR(256),
		YEAR_ONE NUMBER(10,1),
		YEAR_TWO NUMBER(10,1),
		PRIMARY KEY (ID),
);

Adding ddl: 
	CREATE OR REPLACE TRANSIENT TABLE DEADPOOL.PROD.SCORES (
		PLAYER_ID VARCHAR(36),
		TOTAL NUMBER(38,0),
		BASE_SCORE NUMBER(24,1),
		FIRST_BLOOD NUMBER(38,0),
		LAST_BLOOD NUMBER(38,0),
		Q_ONE NUMBER(38,0),
		Q_TWO NUMBER(38,0),
		Q_THREE NUMBER(38,0),
		Q_FOUR NUMBER(38,0),
  		foreign key (PICKED_BY) references DEADPOOL.PROD.PLAYERS(ID)
);
);

Adding docum

'3b0be1acd9b9ccefd0c74ab8ad3afd36-sql'

### If you need to delete all the training data, you can with this

In [7]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,training_data_type,question,content
0,170341-doc,documentation,NaN,The following columns are in the SCORES table ...
1,170340-doc,documentation,NaN,The following columns are in the PLAYERS table...
2,170342-doc,documentation,NaN,The following columns are in the PICKS table i...


In [8]:
for id in training_data['id']:
    vn.remove_training_data(id)

## Ask the Data Some Questions

In [ ]:
vn.ask(question="How many players are there in the game?", visualize=False)

In [ ]:
vn.ask(question="Show the first and last name along with the total number of picks for each player", visualize=True)

In [ ]:
vn.ask(question="Which PLAYER is currently leading the game?", visualize=True)

In [ ]:
vn.ask(question="Show me the scores of the top 3 players in the game?", visualize=True)

## Run a Local Server

In [10]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: off
